In [ ]:
#this cell was made to see if everything's all right
# DO NOT RUN IT
# YOU WILL REGRET IT
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Step 1: Data Loading
data = pd.read_csv("AI_ML_GC/aiml-general-championship/KCDH2024_Training_GroundTruth.csv")

folder_path = "AI_ML_GC/aiml-general-championship/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"
image_paths = [os.path.join(folder_path, filename + '.jpg') for filename in data['image']]
labels = data[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']]
i=0                   
for index, (image_path, row) in enumerate(zip(image_paths, labels.iterrows()), 1):
  #print(image_path)  
    if i==10:
        break
    i+=1               
    if os.path.exists(image_path):
        img = mpimg.imread(image_path)
        plt.imshow(img)
        plt.title(f"Image {index}")
        plt.axis('off')

        # Show disease labels
        diseases = row[1][row[1] == 1].index.tolist()
        plt.text(10, 10, f'Diseases: {", ".join(diseases)}', color='red', fontsize=10, verticalalignment='top')
       # plt.show()
    else:
        print(f"Image file '{image_path}' not found.")


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Step 1: Data Loading and Preprocessing
data = pd.read_csv("AI_ML_GC/aiml-general-championship/KCDH2024_Training_GroundTruth.csv")
folder_path = "AI_ML_GC/aiml-general-championship/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"

image_paths = [os.path.join(folder_path, filename + '.jpg') for filename in data['image']]

images = []
labels = []
for img_path in image_paths:
    if os.path.exists(img_path):
        img = load_img(img_path, target_size=(112,84))
        img = img_to_array(img) / 255.0  # Normalize pixel values
        images.append(img)
        
        # Extract corresponding label from data DataFrame
        filename = os.path.basename(img_path)[:-4]  # Extract filename without extension
        label_row = data[data['image'] == filename]  # Get row corresponding to the image
        label_index = label_row[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values.argmax(axis=1)  # Get index of disease with value 1
        label = label_index[0]  # Extract the disease index
        labels.append(label)
       
# Convert lists to numpy arrays
#images = np.array(images)

images = np.array(images, dtype=np.float16)
labels = np.array(labels)

# Convert labels to one-hot encoded format
labels_one_hot = to_categorical(labels, num_classes=7)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels_one_hot, test_size=0.2, random_state=42)

In [13]:
#VGG Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

input_tensor = Input(shape=(112, 84, 3))  # Custom input shape
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # Output layer with 7 classes

# Combine base model with new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Step 3: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Model Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

Epoch 1/10
240/240 [==============================] - 395s 2s/step - loss: 0.9887 - accuracy: 0.6767 - val_loss: 0.8865 - val_accuracy: 0.7021
Epoch 2/10
240/240 [==============================] - 417s 2s/step - loss: 0.8634 - accuracy: 0.6984 - val_loss: 0.8360 - val_accuracy: 0.7151
Epoch 3/10
240/240 [==============================] - 435s 2s/step - loss: 0.8202 - accuracy: 0.7107 - val_loss: 0.8058 - val_accuracy: 0.7130
Epoch 4/10
240/240 [==============================] - 434s 2s/step - loss: 0.7904 - accuracy: 0.7216 - val_loss: 0.8042 - val_accuracy: 0.7125
Epoch 5/10
240/240 [==============================] - 496s 2s/step - loss: 0.7655 - accuracy: 0.7280 - val_loss: 0.7741 - val_accuracy: 0.7286
Epoch 6/10
240/240 [==============================] - 484s 2s/step - loss: 0.7512 - accuracy: 0.7339 - val_loss: 0.7690 - val_accuracy: 0.7271
Epoch 7/10
240/240 [==============================] - 445s 2s/step - loss: 0.7338 - accuracy: 0.7382 - val_loss: 0.7851 - val_accuracy: 0.7323

In [ ]:
model_save_path = "AI_ML_GC/models_1"
model.save(model_save_path)
print(f"Model saved at {model_save_path}")

import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the path to the folder containing test images
test_folder_path ="AI_ML_GC/aiml-general-championship/KCDH2024_Test_Input/KCDH2024_Test_Input"  # Adjust the path to the folder containing test images

# Preprocess the test images
test_images = []
image_names = []

# Iterate over the images in the test folder
for image_name in os.listdir(test_folder_path):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(test_folder_path, image_name)
        img = load_img(image_path, target_size=(112, 84))
        img = img_to_array(img) / 255.0
        test_images.append(img)
        image_names.append(image_name[:-4])  # Remove the file extension from the image name

test_images = np.array(test_images)

# Load the trained model
model = load_model("AI_ML_GC/models_1")  # Adjust the path to your trained model

# Predict disease labels for the test images
predictions = model.predict(test_images)

# Get the index of the predicted disease (plus 1)
predicted_diseases = np.argmax(predictions, axis=1)

# Create a DataFrame with image names and corresponding disease indices
results_df = pd.DataFrame({'ID': image_names, 'Class': predicted_diseases})

# Save the DataFrame to a CSV file
results_df.to_csv("AI_ML_GC/results/pred_2.csv", index=False)  # Adjust the path where you want to save the results CSV file

In [ ]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from torchvision.transforms import transforms
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array


data = pd.read_csv("AI_ML_GC/aiml-general-championship/KCDH2024_Training_GroundTruth.csv")
folder_path = "AI_ML_GC/aiml-general-championship/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"



# Function to generate image augmentation transformations
def get_aug(args):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_aug = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((args.img_size, args.img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.RandomAdjustSharpness(sharpness_factor=2),
        transforms.RandomErasing(),
        normalize,
    ])
    return train_aug

# Function to load and augment train images for classes with low frequency
def load_and_augment_train_images(train_folder_path, num_augmentations_per_image, low_frequency_classes):
    # Lists to store train images and their corresponding labels
    train_images = []
    train_labels = []
    # Iterate through each image in the train folder
    for image_name in os.listdir(train_folder_path):
        if image_name.endswith('.jpg'):
            image_path = os.path.join(train_folder_path, image_name)
            
            # Load the original image
            img = load_img(image_path, target_size=(112,112))
            img = img_to_array(img) / 255.0
            
            # Apply Gaussian blur for image smoothing
            
            
            # Check the label of the image
            filename = os.path.basename(image_path)[:-4]
            label_row = data[data['image'] == filename]
            label_index = label_row[['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']].values.argmax(axis=1)
            label = label_index[0]
            
            # Append the original image and its label to the lists
            train_images.append(img)
            train_labels.append(label)
            
            # Check if the class of the image is a low frequency class
            # If so, perform data augmentation
            if label in low_frequency_classes:
                augmenter = get_aug(args)
                if label == "MEL" or label == "BKL":
                    num_augmentations_per_image=1;
                if label == "AKIEC":
                    num_augmentations_per_image=2;
                if label == "VASC " or label == "DF":
                     num_augmentations_per_image=4;
                for _ in range(num_augmentations_per_image):
                    augmented_img = augmenter(img_smoothed)
                    train_images.append(augmented_img)
                    train_labels.append(label)
    return np.array(train_images), np.array(train_labels)

# Set parameters
train_folder_path = "AI_ML_GC/aiml-general-championship/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"
num_augmentations_per_image = 3  # Number of augmentations per image
low_frequency_classes =['MEL', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']  # Example low frequency classes (adjust as needed)


# Load and augment train images
train_images, train_labels = load_and_augment_train_images(train_folder_path, num_augmentations_per_image,low_frequency_classes)




In [4]:
train_images = np.array(train_images, dtype=np.float16)
train_labels = np.array(train_labels)

# Convert labels to one-hot encoded format
labels_one_hot = to_categorical(train_labels, num_classes=7)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, labels_one_hot, test_size=0.3, random_state=42)

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
# Step 2: Load MobileNetV2 pre-trained on ImageNet
base_model = MobileNetV2(weights='imagenet', include_top=False)

from tensorflow.keras.layers import Dropout, BatchNormalization

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)  # Add dropout with a dropout rate of 0.5
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
predictions = Dense(7, activation='softmax')(x)

# Combine base model and custom head
model = Model(inputs=base_model.input, outputs=predictions)


# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Step 3: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 4: Model Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
210/210 [==============================] - 97s 409ms/step - loss: 1.2809 - accuracy: 0.6269 - val_loss: 0.9084 - val_accuracy: 0.7083
Epoch 2/10
210/210 [==============================] - 83s 394ms/step - loss: 0.8269 - accuracy: 0.7237 - val_loss: 0.9152 - val_accuracy: 0.7069
Epoch 3/10
210/210 [==============================] - 85s 408ms/step - loss: 0.7470 - accuracy: 0.7368 - val_loss: 0.9180 - val_accuracy: 0.7066
Epoch 4/10
210/210 [==============================] - 84s 400ms/step - loss: 0.7030 - accuracy: 0.7521 - val_loss: 0.8565 - val_accuracy: 0.7104
Epoch 5/10
210/210 [==============================] - 83s 398ms/step - loss: 0.6603 - accuracy: 0.7618 - val_loss: 0.8799 - val_accuracy: 0.7156
Epoch 6/10
210/210 [==============================] - 80s 382ms/step - loss: 0.6186 - accuracy: 0.7792 - val_loss: 0.8733 - val_accuracy: 0.7236
Epoch 7/10
210/210 [==============================] - 81s 386ms/step - loss: 0.5932 - accuracy: 0.7863 - val_loss: 0.8758 - val_ac

In [ ]:

# Define the test folder path
test_folder_path ="AI_ML_GC/aiml-general-championship/KCDH2024_Test_Input/KCDH2024_Test_Input"  # Adjust the path to the folder containing test images



# Lists to store test images and their corresponding names
test_images = []
image_names = []

# Iterate through each image in the test folder
for image_name in os.listdir(test_folder_path):
    if image_name.endswith('.jpg'):
        image_path = os.path.join(test_folder_path, image_name)
        
        # Load the image and resize
        img = load_img(image_path, target_size=(112, 84))
        img = img_to_array(img) / 255.0
        
        # Apply Gaussian blur for image smoothing
        img_smoothed = cv2.GaussianBlur(img, kernel_size, sigma_x)
        
        # Append the smoothed image and its name to the lists
        test_images.append(img_smoothed)
        image_names.append(image_name[:-4])  # Remove the file extension from the image name

# Convert the list of images to a numpy array
test_images = np.array(test_images)

In [6]:
from sklearn.metrics import balanced_accuracy_score

# Assuming you have made predictions on the validation set
y_pred = model.predict(X_val)

# Assuming y_val and y_pred are one-hot encoded arrays
# Convert predictions and ground truth labels back to categorical format
y_val_cat = np.argmax(y_val, axis=1)
y_pred_cat = np.argmax(y_pred, axis=1)

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(y_val_cat, y_pred_cat)

print("Balanced Accuracy:", balanced_acc)

90/90 [==============================] - 24s 248ms/step
Balanced Accuracy: 0.36024739267916284
